In [2]:
import transformers

In [1]:
from transformers import AutoTokenizer
from dataclasses import dataclass
from open_hours_dataset import OpenHoursDataset

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
@dataclass
class open_hours_dataset:
    dataset: str = 'open_hours_dataset'
    train_file: str = "data.jsonl"
    eval_file: str = "data.jsonl"

ds = OpenHoursDataset(tokenizer=tokenizer, config=open_hours_dataset())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
ds[0]

{'input_ids': [128000,
  128006,
  9125,
  128007,
  271,
  2675,
  527,
  264,
  892,
  6373,
  18328,
  16785,
  892,
  5552,
  9256,
  13,
  128009,
  128006,
  882,
  128007,
  271,
  22818,
  264,
  1160,
  315,
  17496,
  10960,
  4207,
  323,
  1063,
  1217,
  26981,
  2737,
  459,
  4839,
  2457,
  323,
  892,
  11,
  8417,
  3508,
  1855,
  315,
  279,
  15926,
  527,
  1825,
  1243,
  627,
  8142,
  3815,
  779,
  11,
  2980,
  279,
  2768,
  512,
  16,
  13,
  364,
  1712,
  1232,
  279,
  5150,
  2457,
  323,
  892,
  304,
  279,
  1217,
  26981,
  198,
  17,
  13,
  364,
  1316,
  3659,
  32377,
  1232,
  279,
  4839,
  1938,
  315,
  279,
  2046,
  430,
  364,
  1712,
  6,
  1053,
  387,
  198,
  18,
  13,
  364,
  68785,
  42471,
  1232,
  279,
  8736,
  4207,
  315,
  279,
  15926,
  304,
  364,
  1316,
  3659,
  32377,
  1270,
  19,
  13,
  364,
  285,
  11563,
  1232,
  264,
  1160,
  315,
  279,
  5144,
  315,
  279,
  15926,
  430,
  527,
  1825,
  304,
  364,
  171

In [4]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [5]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/paperspace/miniconda3/envs/wxg/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/paperspace/miniconda3/envs/wxg/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/paperspace/miniconda3/envs/wxg/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/paperspace/miniconda3/envs/wxg/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/paperspace/miniconda3/envs/wxg/lib/libcudart.so.11.0'), PosixPath('/home/paperspace/miniconda3/envs/wxg/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/home/paperspace/miniconda3/envs/wxg/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
 

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-667ca7d7-2d5400746f4e9fac301c41e6;f9effb90-b68c-43cf-922d-fa5a1d1b5290)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it.

In [28]:
training_args = TrainingArguments(
    output_dir="my_awesome_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.954100,3.832366
2,3.856400,3.822290
3,3.817600,3.821461


TrainOutput(global_step=3933, training_loss=3.878389321178569, metrics={'train_runtime': 3263.0041, 'train_samples_per_second': 9.642, 'train_steps_per_second': 1.205, 'total_flos': 1027582133796864.0, 'train_loss': 3.878389321178569, 'epoch': 3.0})

In [32]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 45.67


VVV SAVE THESE FOR THE MODEL VVV

In [61]:
trainer.model.save_pretrained("my_awesome_clm-model/final")
tokenizer.save_pretrained("my_awesome_clm-model/final")
# trainer.tokenizer.save_pretrained("my_awesome_clm-model/final")


('my_awesome_clm-model/final/tokenizer_config.json',
 'my_awesome_clm-model/final/special_tokens_map.json',
 'my_awesome_clm-model/final/vocab.json',
 'my_awesome_clm-model/final/merges.txt',
 'my_awesome_clm-model/final/added_tokens.json',
 'my_awesome_clm-model/final/tokenizer.json')

In [33]:
prompt = "Somati hypermutation allows the immune system to"

In [63]:
from transformers import pipeline

generator = pipeline("text-generation", model="my_awesome_clm-model/final/")
generator(prompt)

[{'generated_text': "Somati hypermutation allows the immune system to detect if you haven't tried to eat anything at all and they can detect you, too, because your blood also changes over time until you lose sight of a specific symptom. The blood causes a"}]

In [69]:
tokenizer = AutoTokenizer.from_pretrained("my_awesome_clm-model/final")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [70]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("my_awesome_clm-model/final")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [71]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

["Somati hypermutation allows the immune system to take a hit. [Nerve cells in your brain keep making blood flow and then react to certain chemicals. In fact, in the brain, we can tell that many types of cells, like the perihelium we associate with a person, have a strong defense against certain kinds of chemicals. [Your immune system responds to a chemical that has been found in your gut. These chemicals are found in food, blood or plants. [How common is your immune system's response to some of the"]